This recipe follows initially the scikit-learn tutorial on working with text data: https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42
  )

It's a small dataset

In [3]:
len(twenty_train.filenames)

2257

In [4]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


# Using a bag-of-words approach with a classifier

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('clf', RandomForestClassifier()),
])

In [6]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [7]:
import numpy as np
twenty_test = fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=42
)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.8009320905459387

# Using a word embedding with a classifier

In [8]:
# if you use this in colab you can store the embeddings in your google drive. 
# This is optional. 
# If you don't want to do it, just remove the /gdrive part in the next cells
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [23]:
!pip install wget
import os
import wget
filepath = '/gdrive/My Drive/embeddings/wiki.en.vec'
if not os.path.isfile(filepath):
  wget.download(
      'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec',
      filepath
  )

'/gdrive/My Drive/embeddings/wiki.en.vec'

In [25]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format(
    filepath,
    binary=False, encoding='utf8'
)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [26]:
import numpy as np
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def embed_text(text: str):
  vector_list = [
    model.wv[w].reshape(-1, 1) for w in text_to_word_sequence(text)
    if w in model.wv
  ]
  if len(vector_list) > 0:
    return np.mean(
        np.concatenate(vector_list, axis=1),
        axis=1
    ).reshape(1, 300)
  else:
    return np.zeros(shape=(1, 300))


embed_text('training run').shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


(1, 300)

In [27]:
train_transformed = np.concatenate(
    [embed_text(t) for t in twenty_train.data]
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [28]:
train_transformed.shape

(2257, 300)

In [29]:
rf = RandomForestClassifier().fit(train_transformed, twenty_train.target)

In [30]:
test_transformed = np.concatenate(
    [embed_text(t) for t in twenty_test.data]
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [31]:
predicted = rf.predict(test_transformed)
np.mean(predicted == twenty_test.target)

0.8608521970705726

# Keras model with embedding layer

In [32]:
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim=5000, 
    output_dim=50, 
    input_length=500
)

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(twenty_train.data)

In [34]:
X_train = tokenizer.texts_to_sequences(twenty_train.data)
X_test = tokenizer.texts_to_sequences(twenty_test.data)

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import regularizers

model = Sequential()
model.add(embedding)
model.add(layers.Flatten())
model.add(layers.Dense(
    10,
    activation='relu',
    kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
))
model.add(layers.Dense(len(categories), activation='softmax'))
model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 50)           250000    
_________________________________________________________________
flatten (Flatten)            (None, 25000)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                250010    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 44        
Total params: 500,054
Trainable params: 500,054
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(X_train, twenty_train.target, epochs=10)

Epoch 1/10
71/71 [==============================] - 3s 44ms/step - loss: 1.3417 - accuracy: 0.3345
Epoch 2/10
71/71 [==============================] - 4s 60ms/step - loss: 0.7579 - accuracy: 0.7461
Epoch 3/10
71/71 [==============================] - 4s 62ms/step - loss: 0.1984 - accuracy: 0.9730
Epoch 4/10
71/71 [==============================] - 5s 71ms/step - loss: 0.0950 - accuracy: 0.9969
Epoch 5/10
71/71 [==============================] - 5s 65ms/step - loss: 0.0707 - accuracy: 0.9987
Epoch 6/10
71/71 [==============================] - 5s 70ms/step - loss: 0.0583 - accuracy: 0.9996
Epoch 7/10
71/71 [==============================] - 5s 67ms/step - loss: 0.0514 - accuracy: 1.0000
Epoch 8/10
71/71 [==============================] - 5s 74ms/step - loss: 0.0463 - accuracy: 1.0000
Epoch 9/10
71/71 [==============================] - 5s 67ms/step - loss: 0.0422 - accuracy: 1.0000
Epoch 10/10
71/71 [==============================] - 5s 70ms/step - loss: 0.0388 - accuracy: 1.0000


In [38]:
predicted = model.predict(X_test).argmax(axis=1)
np.mean(predicted == twenty_test.target)

0.8695073235685752